# CSE-CIC-IDS 2017 Linear Discriminant Analysis

In [1]:
model_id = "lda1"

In [2]:
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob, pickle, time

In [3]:
NOTEBOOK_PATH = "C:/Users/Xetrov/Desktop/SciFair20/Code/"

In [4]:
x_scaled = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_scaled_powertransform.csv")

y_df_enc = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_all_binary.csv")

In [5]:
x_scaled.shape

(2830743, 68)

# Split data into train and test

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_valtest, y_train, y_valtest = train_test_split(x_scaled, y_df_enc, test_size = 0.4, random_state = 42)

In [8]:
x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size = 0.5, random_state = 42)

In [9]:
del x_train
del y_train

del x_valtest 
del y_valtest

# ADASYN

In [10]:
x_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn_binary.csv")

In [11]:
y_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn_binary.csv")['IsAttack']

# y_train_res2 = pd.get_dummies(y_train_res[y_train_res.columns[0]])

# Train model

In [12]:
print(x_train_res.shape)
print(y_train_res.shape)
print(type(x_train_res))
print(type(y_train_res))

(2727615, 68)
(2727615,)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [13]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [14]:
%%time
clf = LinearDiscriminantAnalysis(n_components=1).fit(x_train_res, y_train_res)

C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Wall time: 27 s


**Save model**

In [15]:
import pickle

import time

model_filename = model_id  + " [" + time.strftime("%Y%m%d %H%M") + "]"

save_file = open(NOTEBOOK_PATH + "Models/" + model_filename + ".pkl", "wb")
saved_model = pickle.dump(clf, save_file)
save_file.close()

**Test model**

In [16]:
pred = clf.predict(x_val)

In [17]:
pred_series = pd.Series(pred, name="Pred").replace({0: 'Benign', 1: 'Attack'})
y_series = pd.Series(y_val.to_numpy().ravel(), name="Actual").replace({0: 'Benign', 1: 'Attack'})

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix

Actual,Attack,Benign,All
Pred,,,
Attack,108513,37844,146357
Benign,3429,416363,419792
All,111942,454207,566149


# Test Set

In [18]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [19]:
pred = clf.predict(x_test)
y_test_npy = y_test.to_numpy().ravel()

precision = precision_score(y_test_npy, pred)
print("Precision:", precision)

recall = recall_score(y_test_npy, pred)
print("Recall:", recall)

f1 = f1_score(y_test_npy, pred)
print("F1:", f1)

Precision: 0.7386176500734566
Recall: 0.9706714320351296
F1: 0.8388926830782604


In [20]:
pred_series = pd.Series(pred, name="Pred").replace({0: 'Benign', 1: 'Attack'})
y_series = pd.Series(y_test.to_numpy().ravel(), name="Actual").replace({0: 'Benign', 1: 'Attack'})

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix

Actual,Attack,Benign,All
Pred,,,
Attack,108093,38252,146345
Benign,3266,416538,419804
All,111359,454790,566149
